In [1]:
import random
import numpy as np
import scipy.interpolate

In [2]:
import os,sys,shutil
### copies a list of files from source. handles duplicates.
def rename(file_name, dst, num=1):
    #splits file name to add number distinction
    (file_prefix, exstension) = os.path.splitext(file_name)
    renamed = "%s(%d)%s" % (file_prefix,num,exstension)

    #checks if renamed file exists. Renames file if it does exist.
    if os.path.exists(dst + renamed):
        return rename(file_name, dst, num + 1)
    else:
        return renamed

def copy_files(src,dst,file_list):
    for files in file_list:
        src_file_path = src + files
        dst_file_path = dst + files
        if os.path.exists(dst_file_path):
            new_file_name =  rename(files, dst)
            dst_file_path = dst + new_file_name
            
        print("Copying: " + dst_file_path)
        try:
            shutil.copyfile(src_file_path,dst_file_path)
        except IOError:
            print(src_file_path + " does not exist")
            raw_input("Please, press enter to continue.")

In [3]:
def get_r1p4(MRL_table):
    max_index = np.argmax(MRL_table[:,0])
    clipped_MRL = MRL_table[:max_index-1,:]
        
    diffMRL = np.ediff1d(clipped_MRL[:,0])
    drop_in_m = np.argmax(diffMRL < 0) # index of first drop in mass
    above_m = np.argmax(clipped_MRL[:,0]>clipped_MRL[drop_in_m,0])
    
    keep = np.zeros(clipped_MRL[:,0].shape[0], dtype=bool)
    for i in range(clipped_MRL[:,0].shape[0]):
        keep[i] = i < drop_in_m or i > above_m
        
    fixed_MRL = clipped_MRL[keep,:]
    r_func = scipy.interpolate.CubicSpline(fixed_MRL[:,0], fixed_MRL[:,1])
    r1p4 = r_func(1.4)
    
    return r1p4

def sorted_r1p4(datapath, MRLdir_name):
    numlist = []
    num_mrl = 0

    for file in os.listdir(datapath+MRLdir_name):
            if '.dat' in file:
                numlist.append(int(file[:-4]))
                num_mrl += 1
    
    r1p4 = np.zeros((num_mrl,2))

    for i in range(num_mrl):
        file = datapath+MRLdir_name+'/'+str(i)+'.dat'
        MRL = np.loadtxt(file)

        r1p4[i,0] = get_r1p4(MRL)
        r1p4[i,1] = i

    sorted_index = np.argsort(r1p4[:,0])
    r1p4_sort = r1p4[sorted_index,:]
    
    return r1p4_sort

In [54]:
datapath = datapath = os.getcwd() + "/data/"
MRLdir_name = "lin5MRL"

r1p4 = sorted_r1p4(datapath, MRLdir_name)

In [55]:
size = r1p4.shape[0]
population = range(0,size)
n_index = 2000
index_list = random.sample(population,n_index)

In [56]:
sorted_dirname = "sorted_" + MRLdir_name
j=0
for i in r1p4[:,1]:
    if i in index_list:
        old_file = datapath+MRLdir_name+'/'+str(int(i))+'.dat'
        old_MRL = np.loadtxt(old_file)
        
        M = old_MRL[:,0]
        R = old_MRL[:,1]
        L = old_MRL[:,2]
        
        MRL = np.zeros(old_MRL.shape)
         
        #radius first
        MRL[:,0] = R
        # then mass
        MRL[:,1] = M
        # finally Lambda
        MRL[:,2] = L
        
        if not(sorted_dirname in os.listdir(datapath)):
            os.makedirs(datapath+sorted_dirname)
        
        new_file = datapath+sorted_dirname+'/'+str(j+1)+'.dat'
        np.savetxt(new_file, MRL)
        j+=1

In [57]:
sorted_datapath = datapath + sorted_dirname + "/"
repopath = "/Users/brendanking/Documents/LANL_Project_eos/"

repo_dirname = MRLdir_name

if not(repo_dirname in os.listdir(repopath)):
    os.makedirs(repopath+repo_dirname)

filelist = os.listdir(sorted_datapath)

In [58]:
copy_files(sorted_datapath, repopath+repo_dirname+"/", filelist)

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1220.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1546.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1552.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/978.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1234.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/950.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/788.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/944.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1208.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/777.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1585.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1591.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/763.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/993.dat

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/760.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1592.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1586.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/774.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/984.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/748.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/990.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1753.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1035.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1021.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1747.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1009.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/589.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/204.dat

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1544.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1222.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1236.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1550.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1568.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/942.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/956.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1554.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1232.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1226.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1540.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/759.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/981.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/995.d

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/969.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1225.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1231.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1557.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/996.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/982.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1580.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/772.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/766.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1594.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1769.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1027.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1999.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1741.d

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1733.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1055.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1069.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1082.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/516.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/270.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/264.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1928.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/502.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1096.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1900.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/258.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1914.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1135.d

...

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1267.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1501.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1515.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1273.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1298.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/718.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/730.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/724.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1059.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/297.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1703.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1065.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1071.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1717.

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/226.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1942.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/568.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1956.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/385.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1611.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1177.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1163.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1605.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/391.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1639.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/346.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/420.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/434.dat

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1160.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1174.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/386.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1612.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1148.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/351.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/437.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/423.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1809.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/345.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1821.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1835.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/379.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1362.da

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/340.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/426.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/5.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/57.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/817.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/43.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/803.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1429.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/195.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1401.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1367.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1373.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1415.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/181.dat
Copyi

Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/6.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1416.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/182.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1370.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/68.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1364.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/828.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/196.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/1402.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/83.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/97.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/169.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/141.dat
Copying: /Users/brendanking/Documents/LANL_Project_eos/lin5MRL/627.dat
Copying